In [ ]:
def deallocate_memory(memory, process, original_memory):
    for i in range(len(memory)):
        if original_memory[i] - memory[i] >= process.memory_required:
            memory[i] += process.memory_required
            break


In [ ]:
class Process:
    def __init__(self, process_id, arrival_time, burst_time, memory_required):
        self.process_id = process_id
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.memory_required = memory_required
        self.remaining_time = burst_time
        self.start_time = None
        self.finish_time = None

import random

def generate_processes(num_processes):
    processes = []
    for i in range(num_processes):
        process_id = i
        arrival_time = random.randint(0, 10)
        burst_time = random.randint(1, 10)
        memory_required = random.randint(1, 100)
        processes.append(Process(process_id, arrival_time, burst_time, memory_required))
    return processes

def fcfs(processes):
    processes.sort(key=lambda x: x.arrival_time)
    current_time = 0
    for process in processes:
        if current_time < process.arrival_time:
            current_time = process.arrival_time
        process.start_time = current_time
        process.finish_time = current_time + process.burst_time
        current_time += process.burst_time

def sjf(processes):
    processes = sorted(processes, key=lambda x: (x.arrival_time, x.burst_time))
    current_time = 0
    completed_processes = []

    while processes:
        for i, process in enumerate(processes):
            if process.arrival_time <= current_time:
                current_time += process.burst_time
                process.finish_time = current_time
                completed_processes.append(process)
                processes.pop(i)
                break
        else:
            current_time += 1

    processes.extend(completed_processes)

def round_robin(processes, time_quantum):
    queue = processes[:]
    current_time = 0
    while queue:
        process = queue.pop(0)
        if process.remaining_time > time_quantum:
            process.remaining_time -= time_quantum
            current_time += time_quantum
            queue.append(process)
        else:
            current_time += process.remaining_time
            process.remaining_time = 0
            process.finish_time = current_time

def first_fit(memory, process):
    for i in range(len(memory)):
        if memory[i] >= process.memory_required:
            memory[i] -= process.memory_required
            return True
    return False

def best_fit(memory, process):
    best_index = -1
    best_size = float('inf')
    for i in range(len(memory)):
        if memory[i] >= process.memory_required and memory[i] < best_size:
            best_size = memory[i]
            best_index = i
    if best_index != -1:
        memory[best_index] -= process.memory_required
        return True
    return False

def deallocate_memory(memory, process, original_memory):
    for i in range(len(memory)):
        if original_memory[i] - memory[i] >= process.memory_required:
            memory[i] += process.memory_required
            break

def calculate_metrics(processes):
    total_waiting_time = 0
    total_turnaround_time = 0
    n = len(processes)
    for process in processes:
        turnaround_time = process.finish_time - process.arrival_time
        waiting_time = turnaround_time - process.burst_time
        total_turnaround_time += turnaround_time
        total_waiting_time += waiting_time

    avg_waiting_time = total_waiting_time / n
    avg_turnaround_time = total_turnaround_time / n
    print(f"Average Waiting Time: {avg_waiting_time}")
    print(f"Average Turnaround Time: {avg_turnaround_time}")

def simulate(processes, scheduling_algorithm, memory_management, memory_size):
    original_memory = [memory_size]
    memory = original_memory.copy()
    copied_processes = [Process(p.process_id, p.arrival_time, p.burst_time, p.memory_required) for p in processes]

    scheduling_algorithm(copied_processes)

    for process in copied_processes:
        if not memory_management(memory, process):
            print(f"Process {process.process_id} cannot be allocated memory")
        else:
            print(f"Process {process.process_id} is allocated memory")
        deallocate_memory(memory, process, original_memory)

    calculate_metrics(copied_processes)




In [ ]:
def test_scheduling_and_memory():
    processes = [
        Process(0, 0, 4, 10),
        Process(1, 1, 3, 20),
        Process(2, 2, 1, 30)
    ]

    print("Testing FCFS:")
    simulate(processes, fcfs, first_fit, 100)

    processes = [
        Process(0, 0, 4, 10),
        Process(1, 1, 3, 20),
        Process(2, 2, 1, 30)
    ]

    print("Testing SJF:")
    simulate(processes, sjf, best_fit, 100)

    processes = [
        Process(0, 0, 4, 10),
        Process(1, 1, 3, 20),
        Process(2, 2, 1, 30)
    ]

    print("Testing RR:")
    simulate(processes, lambda p: round_robin(p, 2), first_fit, 100)

test_scheduling_and_memory()

Testing FCFS:
Process 0 is allocated memory
Process 1 is allocated memory
Process 2 is allocated memory
Average Waiting Time: 2.6666666666666665
Average Turnaround Time: 5.333333333333333
Testing SJF:
Process 0 is allocated memory
Process 1 is allocated memory
Process 2 is allocated memory
Average Waiting Time: 2.6666666666666665
Average Turnaround Time: 5.333333333333333
Testing RR:
Process 0 is allocated memory
Process 1 is allocated memory
Process 2 is allocated memory
Average Waiting Time: 3.0
Average Turnaround Time: 5.666666666666667
